# Stationary Harmonic Oscillator

#### In this section we begin by creating a generalized function for performing the Euler-Cromer algorithm given a set of data. We also create a function for calculating the energies that satisfy boundary conditions for more complicated examples. We use these functions to solve for the harmonic and anharmonic oscillator (in the stationary sense). 

Remark: we are working in natural units


Import necessary libraries

In [ ]:
import numpy as np
import  matplotlib.pyplot as plt
from typing import Callable
from scipy import special
import ipywidgets as wid
plt.rcParams['figure.figsize'] = [10, 5]
%matplotlib qt

We define the Euler Cromer algorithm by giving it the following variables:
- `y_0`: the initial value for $\psi_0(x=0)$
- `dy_0`: the initial value for $\frac{\partial }{\partial x} \psi_0(x)\biggr\rvert_{x=0}$
- `E`: The energy we assume to be an eigenvalue of the hamiltonian $\hat{H}$
- `V`: The function $V(x)$ that takes a value over $x$ (float) and outputs a float
- `x_f`: The range where the calculations are computed ( $x\in [x_f, x_f]$ )
- `del_x`: The interval between each neighboring point in $x$-space after discretizing ( $\Delta x$ )

Since our hamiltonian is given by 

$$\hat{H} = -\frac{1}{2}\frac{\partial^2}{\partial x^2} + V(x)$$

the Time Independent SE $\hat{H} \psi(x) = E \, \psi(x)$ provides the equation:

$$\psi_0'' (x) = -2(E - V(x)) \psi(x)$$

If we discretize and use a semi-implicit algorithm (Euler-Cromer), the following equations follow:

* $\psi_0''(x_{n+1}) = -2(E-V(x_n))\psi_0(x_n)$
* $\psi_0'(x_{n+1}) = \psi_0'(x_{n})+ \psi_0''(x_{n}) \cdot \Delta x$
* $\psi_0(x_{n+1}) = \psi_0(x_{n})+ \psi_0'(x_{n+1}) \cdot \Delta x$

With $x_k \coloneqq k\cdot \Delta x \equiv$ `k*d_x`


In [ ]:
def euler_cromer(y_0: float, dy_0: float, E: float,
                 V: Callable[[float], float], x_f: int, del_x: float):
    n = int(2*x_f/del_x)
    xs = np.linspace(-x_f, x_f, n)
    psi = np.zeros(n)
    d_psi = np.zeros(n)
    y = y_0
    dy = dy_0
    for idx, x in enumerate(xs):
        # if abs(y) >= 30000:
        #     break
        psi[idx] = y
        d_psi[idx] = V(x)
        ddy = -2*(E - V(x))*y
        dy = dy + del_x*ddy
        y = y + dy*del_x
    rho = psi*np.conjugate(psi)
    psi = psi/np.sqrt(np.sum(rho)*del_x)
    return xs, psi, d_psi

Since the Harmonic oscillator has an analytic solution ( $E_n = \frac{1}{2} +n$ ), our "educated guesses" will be set to these values. We proceed to use the Euler-Cromer algorithm to numerically compute the eigenstates, which we wil compare to the analytic solutions:
$$\phi_n(x) = H_n e^{\frac{-x^2}{2}}$$

Note: We need to use different initial conditioins depending on the parity of the quantum number.

In [ ]:

Es = np.arange(0.5, 11, 1)
xlim, dx = 4, 0.001
for e in Es:
    plt.figure()
    n = int(e - 0.5)
    if not (n%2): 
        """Even Parity"""
        x, y , dy= euler_cromer(0, 1, e, lambda x: x**2/2, xlim, dx)
    else: 
        """Odd Parity"""
        x, y, dy = x, y , dy= euler_cromer(0, 1, e, lambda x: x**2/2, xlim, dx)
        y = -y  # Psi only defined up to a phase
    plt.plot(x, y, label=r'$\Psi_{num}$')  
    eigenfunc = lambda x: (
        special.hermite(n=n).__call__(x)*np.exp(-x**2/2)/np.sqrt(
            2**n * np.math.factorial(n)*np.sqrt(np.pi)))
    plt.plot(x, eigenfunc(x), ls='--', label=r'$\Psi_{theory}$')
    plt.xlabel(r'$x$')
    plt.ylabel(r'$\Psi(x)$')
    plt.title(f'Wavefunction with $E = {e}$')
    plt.grid(ls='--')
    plt.legend()
    plt.show()

We can see in these results that, for lower energies, the method works as expected and aligns almost perfectly with the theoretical eigenstates, but for the deviations become significant for the later eigenvalues.

### $\alpha x^4$
Now we try for the anharmonic oscillator with $V(x) = \frac{1}{2}x^2+\alpha x^4$. We can see that the behaviour, in comparison to the anharmonic oscillator, is very similar. Note, however, that the anharmonic oscillator will diverge if the same eigenenergies for the harmonic oscillator are used, so it is important to find the right eigenvalues. 

The method used to find these enegies is creating using small variations and utilizing the ones that produce a function that does not diverge (and has an end result of $0$). Then we repeat the same procedure as before to plot the eigenstates corresponding to these energies.

In [ ]:
alpha = 0.01
Es = np.arange(0.1, 10, 0.1)

boundaries = np.zeros(len(Es))
for i, e in enumerate(Es):
    n = int(e - 0.5)
    x, y , dy= euler_cromer(0, 1, e, lambda x: x**2/2+alpha*x**4, xlim, dx)
    boundaries[i] = y[-1]
boundaries = boundaries*np.conjugate(boundaries)
plt.figure()
plt.xlabel(r'$E$')
plt.ylabel(r'$\psi(x_{lim})$')
plt.title(f'Wavefunction at boundary as a function of energy')
plt.plot(Es, boundaries)
k = 5
k_lowest = np.argpartition(boundaries, k)[:k].tolist()
for idx in k_lowest:
    plt.scatter(Es[idx], boundaries[idx], color='r', marker='x')
    
for idx in k_lowest:
    plt.figure()
    e = Es[idx]
    n = int(e - 0.5)
    x, y , dy= euler_cromer(0, 1, e, lambda x: x**2/2+alpha*x**4, xlim, dx)
    _, y2, dy2 = euler_cromer(0, 1, e, lambda x: x**2/2, xlim, dx)

    plt.title(f'$E = ${e}')
    plt.plot(x, y,  label=f'$V(x) = x^2+{alpha}x^4$')
    plt.plot(x, y2, label=f'$V(x) = x^2$', ls='--')
    plt.grid(ls='--')
    plt.xlabel(r'$x$')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()
    plt.show()




# 2. Propagating Wavepacket

#### In this section, we will analyze the dynamics of different systems. In order to do this, we will employ a variety of tools (such as Fourier transforms) in order to represent different operators in the most convenient ways.

In [ ]:
from scipy import fft
from matplotlib import animation
from IPython import display
from time import sleep
sleep(4)

In [ ]:
# xlim, dx = 10, 0.1
# xs = np.arange(-xlim, xlim+dx, dx)
# N = len(xs)
# psi0 = (np.pi)**0.25*np.exp(-xs**2/2)
# plt.figure()
# plt.plot(xs, psi0)
# plt.show()
# psi_tilde = fft.fft(psi0)
# ps = fft.fftfreq(N, dx)
# plt.figure()
# plt.plot(ps, psi_tilde.real)
# plt.show()

We begin by defiing a function that carries out the time evolution of a free particle. This algorithm stems from the time propagation equation:

$$ \psi(t) = e^{-i\hat{H}t}\psi_0$$

We can divide a time intertval into infinitesimal time-steps, which (after performing a delicate analysis on the communation of operators), we can obtain the time evolved states by iterating over the action of $e^{-i\hat{H}\Delta t}$.

Given that for a free particle $\hat{H} = \frac{1}{2}\hat{p}^2$, the time evolution results in the following algorithm:

In [ ]:
def free_time_evo(psi: np.ndarray, dx: float,
                  N: int, dt: float, M: int, interv: int) -> np.ndarray:
    plots = []
    fftpsi = fft.fft(psi)
    ps = fft.fftfreq(n=N, d=dx)
    for m in range(M):
        if m%interv == 0:
            temp = fft.ifft(fftpsi)
            plots.append(temp)

        fftpsi = np.exp(-1j*dt*ps**2/2)*fftpsi
    return plots



_Remark_: The plots extracted from this procedure will not ocurr for each frame, but rather for one frame every interval of length `interv` has passed.

### Free wave
Now we define the initial state as $\psi_0 = \pi^{-\frac{1}{4}} e^{-\frac{x}{2}}$. And using `FuncAnimate()`, we animate the time evolution of this state. As we can see, this function behaves as expected, by simply dispersing as time evolves (and eventually creating some interference as it reaches the boundary).

In [ ]:

xlim, dx = 10, 0.1
dt, M = 0.1, 3000
xs = np.arange(-xlim, xlim+dx, dx)
N = len(xs)
psi0 = (np.pi)**0.25*np.exp(-xs**2/2)
fig, ax = plt.subplots()
l, = ax.plot([],[])
ax.set_xlim([-xlim, xlim])
ax.set_ylim([-0.5, 1.5])
ax.grid(ls='--')
ax.set_title('Free wave propagation')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$|\Psi(x)|^2$')
numFrames = 100

plots = free_time_evo(psi0, dx, N, dt, M, M/numFrames)


def func(frame, *fargs):
    y = fargs[0][frame]
    x = fargs[1]
    l.set_data(x, np.absolute(y))
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$|\Psi(x)|^2$')
    ax.set_title('Free wave propagation') 

anim = animation.FuncAnimation(fig, func, fargs=[plots, xs],
                               frames=numFrames, interval=40)
display.HTML(anim.to_jshtml())

We repeat the same procedure, but now giving the particle an initial 'kick' with momentum $k_0 = 1$, which we add through an exponential phase to our $\psi_0$. We will find that the particle travels in the direction given by the momentum (and disperses), just as expected.

In [ ]:
k0 = 1
xlim, dx = 40, 0.1
xs = np.arange(-xlim, xlim+dx, dx)
N = len(xs)
psi0_moving = (np.pi)**0.25*np.exp(-xs**2/2)*np.exp(1j* k0*xs)
fig, ax = plt.subplots()
l, = ax.plot([],[])
ax.set_xlim([-xlim, xlim])
ax.set_ylim([-0.5, 1.5])
ax.grid(ls='--')
ax.set_title('Free wave propagation')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$|\Psi(x)|^2$')
numFrames = 100
dt, M = 0.1, 3000

plots = free_time_evo(psi0_moving, dx, N, dt, M, M/numFrames)
def func(frame, *fargs):
    y = fargs[0][frame]
    x = fargs[1]
    l.set_data(x, np.absolute(y))

anim = animation.FuncAnimation(fig, func, fargs=[plots, xs],
                               frames=numFrames, interval=40)
display.HTML(anim.to_jshtml())

We then proceed to define different time evolution operators, which include a potential term. In order to do this, we will 
- Once again split the exponential 
- Act with the potential as a function of x 
- Perform a fourier transform to act with the momentum squared on momentum space
- And then transform back to obtain our wavefunction in x-space. 

(Rather, we will divide the potential term intwo two and act on it at the beginning of the procedure and then int the end, as this can be more stable in some situations).

Overall, the operators that act on the wavefunction are:

$$e^{-\frac{1}{2}i\hat{V}(x)\Delta t} \circ \mathscr{F}^{-1} \circ e^{-\frac{1}{2}i\hat{p}^2\Delta t} \circ \mathscr{F} \circ e^{-\frac{1}{2}i\hat{V}(x)\Delta t} 

In [ ]:
def potential_time_evo(psi: np.ndarray, dx: float, N: int, 
                       dt: float, M: int, interv: int, V_x: np.ndarray) -> np.ndarray:
    plots = []
    mu_x = []
    mu_k = []
    
    xs = np.arange(-xlim, xlim+dx, dx)
    ps = 2*np.pi*fft.fftfreq(n=N, d=dx)
    
    for m in range(M):
        if m%interv == 0:
            plots.append(psi)
        psi = np.exp(-1j*V_x*dt/2)*psi
        fftpsi = fft.fft(psi)
        fftpsi = np.exp(-1j*dt*ps**2/2)*fftpsi
        psi = fft.ifft(fftpsi)
        psi = np.exp(-1j*V_x*dt/2)*psi
        mu_x.append(np.dot(xs*psi,np.conjugate(psi))/N)
        mu_k.append(np.dot((ps*fftpsi),np.conjugate(fftpsi))/N)
    return plots, np.array(mu_x), np.array(mu_k)

### Potential Barrier

Now we proceed to analyze the potential barrier, by setting up the interval for which we want a potential barrier to be defined. We will juxtapose the potential with the wavefunction. We can see that the wavefunction is partially transmitted (tunneling) and partially reflected, which acts according to theory.

In [ ]:
k0 = 5
xlim, dx = 50, 0.1
xs = np.arange(-xlim, xlim+dx, dx)
N = len(xs)
psi0_moving = (np.pi)**(-1/4)*np.exp(-xs**2/2)*np.exp(1j* k0*xs)


V, V0 = np.zeros(N), 5
u1, u2 = int(N/2+xlim*0.2/dx), int(N/2 + xlim*0.21/dx)
for i in range(u1, u2+1):
    V[i] = V0
    

fig, ax = plt.subplots()
l, = ax.plot([],[])
ax.set_xlim([-xlim, xlim])
ax.set_ylim([-0.5, 1.5])
ax.grid(ls='--')
ax.set_title('Potential Barrier')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$|\Psi(x)|^2$')
numFrames = 50
dt, M = 0.01, 600

plots,_, _ = potential_time_evo(psi0_moving, dx, N, dt, M, M/numFrames, V)
plt.plot(xs, V)
def func(frame, *fargs):
    y = fargs[0][frame]
    x = fargs[1]
    l.set_data(x, np.absolute(y))

anim = animation.FuncAnimation(fig, func, fargs=[plots, xs],
                               frames=numFrames, interval=40)
display.HTML(anim.to_jshtml())

### Quantum Harmonic Oscillator

We now add the harmonic oscillator potential, and give some initial momentum to the particle. This function is a coherent state, so we expect an oscillatory behaviour from the wavefunction.

Thus, we will also perform an analysis on the expectation values $\bra{\psi(t)}\hat{p}\ket{\psi(t)}$ (up to a constant) and $\bra{\psi(t)}\hat{x}\ket{\psi(t)}$.

In [ ]:
k0 = 2
xlim, dx = 10, 0.1
xs = np.arange(-xlim, xlim+dx, dx)
N = len(xs)
psi0_moving = (np.pi)**(-1/4)*np.exp(-xs**2/2)*np.exp(1j* k0*xs)

V = xs**2/2
fig, ax = plt.subplots()
l, = ax.plot([],[])
ax.set_xlim([-xlim, xlim])
ax.set_ylim([-0.5, 1.5])
ax.grid(ls='--')
ax.set_title('Quantum Harmonic Oscillator')
ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$|\Psi(x)|^2$')
numFrames = 100
dt, M = 0.01, 1000
t = np.linspace(0, dt*M, M)

plots,mu_x, mu_k = potential_time_evo(psi0_moving, dx, N, dt, M, M/numFrames, V)

def func(frame, *fargs):
    y = fargs[0][frame]
    x = fargs[1]
    l.set_data(x, np.absolute(y))
plt.plot(xs, V)
anim = animation.FuncAnimation(fig, func, fargs=[plots, xs],
                               frames=numFrames, interval=40)
display.HTML(anim.to_jshtml())



We see that the behaviour for the expectation values is just as we expected from the classical analogue of the system. Thus, this implies that the code reflects a proper description of the systems at hand..

In [ ]:
plt.figure()
plt.ylabel(r'$\mu(\hat{x})_\Psi$')
plt.xlabel(r'$t$')
plt.plot(t, mu_x.real)
plt.grid(ls='--')
plt.title('Expectation value of position over time')
plt.show()

plt.figure()
plt.ylabel(r'$\mu(\hat{k})_\Psi$')
plt.xlabel(r'$t$')
plt.grid(ls='--')
plt.plot(t, mu_k.real)
plt.title('Expectation value of momentum vs time')
plt.show()